In [1]:
import numpy as np
import torch
from PIL import Image
from torch.nn import functional as F
from tqdm import tqdm
import onnx
import onnxruntime
from onnx import numpy_helper
import os
import time

In [2]:
################### float32数据转int8模块  ##############
def float32_to_int8(float32_array): 
   # 定义量化范围（最小和最大值）
    min_val = np.min(float32_array)
    max_val = np.max(float32_array)

    # 计算缩放因子和零点
    scale = (max_val - min_val) / 127.0  # 127 是有符号 int8 的最大值
    zero_point = np.round(0 - min_val / scale)

    # 使用缩放因子和零点进行量化
    int8_array = np.round(float32_array / scale + zero_point).astype(np.int8)

    return int8_array   
    

In [3]:
def bitlet_time(arr_w, arr_a):
    arr_w=float32_to_int8(arr_w)
    arr_a=float32_to_int8(arr_a)

    mantis = np.empty((8,64))
    time = 0

    weight_int8 = [""] * 64
    active_int8 = [""] * 64
    a_sign = [""] * 64
    w_sign = [""] * 64
    sign   = [""] * 64
                            
    sign = [""] * 64
    for b in range(64): 
        weight_int8[b] = '{0:08b}'.format(abs(arr_w[b]))
        active_int8[b] ='{0:08b}'.format(abs(arr_a[b]))   
        
        if arr_w[b] >= 0:
                w_sign[b] = 0
        else:
                w_sign[b] = 1

        if arr_a[b] >= 0:
                a_sign[b] = 0
        else:
                a_sign[b] = 1
                
        if w_sign[b]^a_sign[b]:
                sign[b] = 1
        else:
                sign[b] = 0
                
    w_manti_list = []
    a_manti_list = []

    for c in range(64):
            w_manti_list.append(weight_int8[c])
            a_manti_list.append(active_int8[c])


    
    for i in range(64):
        for j in range(8):
            mantis[j][i] = w_manti_list[j]


    ones = np.empty((8,4))
    for i in range(8):
        for k in range(4):
            one = 0
            for j in range(8):
                if mantis[i][32+8*k+j]:
                    one = one + 1 
            ones[i][k] = one

    for i in range(8):
        for k in range(4):
            if (ones[i][k] == 0 or ones[i][k] == 1):
                ones[i][k] = k+6
            elif (ones[i][k] == 2):
                ones[i][k] = k+7
            elif (ones[i][k] == (3 or 4)):
                ones[i][k] = k+8
            else:
                ones[i][k] = k+9

    for i in range(8):
        for k in range(3):
            for j in range(3-k):
                if(ones[i][k] == ones[i][k+j+1]):
                    ones[i][k] = ones[i][k] + 1

    for i in range(8):
        for k in range(3):
            for j in range(3-k):
                if(ones[i][k] == ones[i][k+j+1]):
                    ones[i][k] = ones[i][k] + 1

    for i in range(8):
        for k in range(3):
            for j in range(3-k):
                if(ones[i][k] == ones[i][k+j+1]):
                    ones[i][k] = ones[i][k] + 1

    time = ones[0][0]
    for i in range(8):
        for k in range(4):
            if(time < ones[i][k]):
                time = ones[i][k]


    
    
    return time+5

In [4]:

# def Mac_64(arr_w, mac_a64):
#     weight_size = int(arr_w.size/10)
#     mac_time = 0

#     for i in range(0, weight_size, 64):
#         mac_time = mac_time + bitlet_time(arr_w[i:i+64],mac_a64)
#     return mac_time

def Mac_64(arr_w, mac_a64):
    weight_size = int(arr_w.size/10)
    mac_time = 0

    for i in range(0, weight_size, 64):
        # print(bitlet_time(arr_w[i:i+64],mac_a64))
        mac_time = mac_time + 1
    return mac_time


def Conv_33(arr_w, arr_a, w_shape, a_shape, s):

    if(arr_w.size % 64 != 0):
        arr_w = np.pad(arr_w,(0,(64 - arr_w.size % 64)))

    yolo_time = 0
    mac_a64 = np.empty(64)
    cnt = 0

    for i in range(0 , (a_shape[3] - 2), s): 
        print("row = ",i)           #row
        for j in range(0 , (a_shape[2] - 2), s):        #channel
            for k in range(a_shape[1]):        #col
                for c in range(3):
                    for r in range(3):
                        mac_a64[cnt] =  arr_a[i+c,j+r,k]
                        cnt = cnt + 1
                        if(cnt == 64):
                            cnt = 0
                            yolo_time = yolo_time + Mac_64(arr_w, mac_a64)
            print(j , end = "|")

    return yolo_time


def Conv_11(arr_w, arr_a, w_shape, a_shape):

    if(arr_w.size % 64 != 0):
        arr_w = np.pad(arr_w,(0,(64 - arr_w.size % 64)))

    yolo_time = 0
    mac_a64 = np.empty(64)
    cnt = 0

    for i in range(a_shape[3]):
        print("row = ",i)            #row
        for j in range(a_shape[2]):        #channel
            for k in range(a_shape[1]):        #col
                mac_a64[cnt] =  arr_a[i,j,k]
                cnt = cnt + 1
                if(cnt == 64):
                    cnt = 0
                    yolo_time = yolo_time + Mac_64(arr_w, mac_a64)
            print(j , end = "| ")

    return yolo_time

In [5]:
import numpy as np
import torch
from PIL import Image
from torch.nn import functional as F
from tqdm import tqdm
import onnx
import onnxruntime
from onnx import numpy_helper
import os
import time

def extract_onnx_weight(onnx_path):
    model = onnx.load(onnx_path)

    weights, names = [], []
    for t in model.graph.initializer:
        weights.append(numpy_helper.to_array(t))
        names.append(t.name)
        
    onnx_weight = dict()
    for name, weight in zip(names, weights):
        onnx_weight[name] = weight
    return onnx_weight

if __name__ == '__main__':


    cwd = os.getcwd()
    print(cwd)
    IMG_PATH = 'yolov5_fp32/dog_cat.jpg'
    ONNX_PATH = "yolov5_fp32/yolov5n_quant_0505.onnx"
    OUT_TXT_PATH = "yolov5_fp32/data.txt"
    
    #WEIGHT_NAME = "1134"
    #MED_LAYER_NAME = "592"
    # w = ["1142","1146","1150"]           #手动输入onnx的w
    # a = ["606", "614", "621"]             #手动输入onnx的x
    w = ["backbone1.conv.weight_quantized", "backbone2.cv1.conv.weight_quantized", "backbone2.m.0.cv1.conv.weight_quantized", "backbone2.m.0.cv2.conv.weight_quantized", "backbone2.cv2.conv.weight_quantized", "backbone2.cv3.conv.weight_quantized", "backbone3.conv.weight_quantized", "backbone4.cv1.conv.weight_quantized", "backbone4.m.0.cv1.conv.weight_quantized", 
"backbone4.m.0.cv2.conv.weight_quantized", "backbone4.m.1.cv1.conv.weight_quantized", "backbone4.m.1.cv2.conv.weight_quantized", "backbone4.cv2.conv.weight_quantized", "backbone4.cv3.conv.weight_quantized", "backbone5.conv.weight_quantized", "backbone6.cv1.conv.weight_quantized", "backbone6.m.0.cv1.conv.weight_quantized", "backbone6.m.0.cv2.conv.weight_quantized", "backbone6.m.1.cv1.conv.weight_quantized", 
"backbone6.m.1.cv2.conv.weight_quantized", "backbone6.m.2.cv1.conv.weight_quantized", "backbone6.m.2.cv2.conv.weight_quantized", "backbone6.cv2.conv.weight_quantized", "backbone6.cv3.conv.weight_quantized", "backbone7.conv.weight_quantized", "backbone8.cv1.conv.weight_quantized", "backbone8.m.0.cv1.conv.weight_quantized", "backbone8.m.0.cv2.conv.weight_quantized", "backbone8.cv2.conv.weight_quantized", 
"backbone8.cv3.conv.weight_quantized", "backbone9.cv1.conv.weight_quantized", "backbone9.cv2.conv.weight_quantized", "neck0.conv.weight_quantized", "neck3.cv1.conv.weight_quantized", "neck3.m.0.cv1.conv.weight_quantized", "neck3.m.0.cv2.conv.weight_quantized", "neck3.cv2.conv.weight_quantized", "neck3.cv3.conv.weight_quantized", "neck4.conv.weight_quantized", 
"neck7.cv1.conv.weight_quantized", "neck7.m.0.cv1.conv.weight_quantized", "neck7.m.0.cv2.conv.weight_quantized", "neck7.cv2.conv.weight_quantized", "neck7.cv3.conv.weight_quantized", "neck8.conv.weight_quantized", "neck10.cv1.conv.weight_quantized", "neck10.m.0.cv1.conv.weight_quantized", "neck10.m.0.cv2.conv.weight_quantized", "neck10.cv2.conv.weight_quantized", 
"neck10.cv3.conv.weight_quantized", "neck11.conv.weight_quantized", "neck13.cv1.conv.weight_quantized", "neck13.m.0.cv1.conv.weight_quantized", "neck13.m.0.cv2.conv.weight_quantized", "neck13.cv2.conv.weight_quantized", "neck13.cv3.conv.weight_quantized", "head2.weight_quantized" ]           #手动输入onnx的w
   
    a = [ "123_quantized", "126_quantized", "129_quantized", "132_quantized", "126_quantized", "140_quantized", "143_quantized", "146_quantized", "149_quantized", 
"152_quantized", "156_quantized", "159_quantized", "146_quantized", "167_quantized", "170_quantized", "173_quantized", "176_quantized", "179_quantized", "183_quantized", 
"186_quantized", "190_quantized", "193_quantized", "173_quantized", "201_quantized", "204_quantized", "207_quantized", "210_quantized", "213_quantized", "207_quantized", 
"221_quantized", "224_quantized", "231_quantized", "234_quantized", "243_quantized", "246_quantized", "249_quantized", "243_quantized", "256_quantized", "259_quantized", 
"268_quantized", "271_quantized", "274_quantized", "268_quantized", "281_quantized", "284_quantized", "288_quantized", "291_quantized", "294_quantized", "288_quantized", 
"301_quantized", "304_quantized", "308_quantized", "311_quantized", "314_quantized", "308_quantized", "321_quantized", "324_quantized"]
    s = [ 2,1,1,1,1,1,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1]
    
    print(len(w))
    print(len(a))
    print(len(s))
    torch.manual_seed(999)
    torch.set_printoptions(precision=10,sci_mode=True)
    torch.set_printoptions(threshold=np.inf)

    # activation
    img = Image.open(IMG_PATH).convert("RGB")
    img = np.asarray(img, np.float32)/255.0
    img= img.transpose(2,0,1).reshape(1,3,224,224)
    

    # weight
    # onnx_path_list = os.listdir(ONNX_PATH)
    onnx_path_list = [ONNX_PATH]
    
    out_file = OUT_TXT_PATH
    if os.path.exists(out_file):
        os.remove(out_file)
        print("文件已存在并已删除。")
    else:
        print("文件不存在。")

    final_time = np.empty((100))
    compute_time = np.empty((100))

    for layerX in range(len(w)):
        start_time = time.time()
        print("This is layer "+ str(layerX))
        #f = open(out_file, 'a', encoding='utf-8')

        print(w[layerX])
        WEIGHT_NAME = str(w[layerX])        #输入w
        MED_LAYER_NAME = str(a[layerX])      #输入a

        onnx_name = "yolov5n_fuse_test.onnx"
        yolov3_weight = extract_onnx_weight(ONNX_PATH)
        select_weight = WEIGHT_NAME # "1138"
        weight = torch.from_numpy(yolov3_weight[select_weight])

        # activate
        intermediate_layer_name = MED_LAYER_NAME  
        model = onnx.load(ONNX_PATH)
        for node in model.graph.node:
            for output in node.output:
                if(output==intermediate_layer_name):
                    model.graph.output.extend([onnx.ValueInfoProto(name=output)])
        session = onnxruntime.InferenceSession(model.SerializeToString())
        # model input
        inputs = {'input.1': img}
        # infer
        intermediate_layer_output = session.run([intermediate_layer_name], inputs)[0]

        act = torch.from_numpy(intermediate_layer_output)

        ######################################################################
        w_shape = weight.shape 
        a_shape = act.shape

        print('w_shape:',w_shape)
        print('a_shape:',a_shape)
        core_size = w_shape[3]

        weight = weight.numpy().reshape(-1)
        act = act.numpy().reshape(a_shape[1],a_shape[2],a_shape[3]).transpose(1,2,0)
        print('after modify:')
        print('w_shape:',weight.shape)
        print('a_shape:',act.shape)

        #####################################################################
        if(core_size == 3 ):
            f_time = Conv_33(weight, act, w_shape, a_shape, s[layerX])
        else:
            f_time = Conv_11(weight, act, w_shape, a_shape)
        end_time = time.time()
        compute_time[layerX] =end_time - start_time 
        final_time[layerX] = f_time
        print(f_time)
        print("cp_time",end_time - start_time)

/home/chen/ROCC/python/mac_test/mac_test_yolov5
57
57
57
文件不存在。
This is layer 0
backbone1.conv.weight_quantized
w_shape: torch.Size([32, 16, 3, 3])
a_shape: torch.Size([1, 16, 112, 112])
after modify:
w_shape: (4608,)
a_shape: (112, 112, 16)
row =  0
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|38|40|42|44|46|48|50|52|54|56|58|60|62|64|66|68|70|72|74|76|78|80|82|84|86|88|90|92|94|96|98|100|102|104|106|108|row =  2
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|38|40|42|44|46|48|50|52|54|56|58|60|62|64|66|68|70|72|74|76|78|80|82|84|86|88|90|92|94|96|98|100|102|104|106|108|row =  4
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|38|40|42|44|46|48|50|52|54|56|58|60|62|64|66|68|70|72|74|76|78|80|82|84|86|88|90|92|94|96|98|100|102|104|106|108|row =  6
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|38|40|42|44|46|48|50|52|54|56|58|60|62|64|66|68|70|72|74|76|78|80|82|84|86|88|90|92|94|96|98|100|102|104|106|108|row =  8
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|3

/tmp/ipykernel_82979/4143570322.py:92: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  weight = torch.from_numpy(yolov3_weight[select_weight])


92|94|96|98|100|102|104|106|108|row =  48
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|38|40|42|44|46|48|50|52|54|56|58|60|62|64|66|68|70|72|74|76|78|80|82|84|86|88|90|92|94|96|98|100|102|104|106|108|row =  50
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|38|40|42|44|46|48|50|52|54|56|58|60|62|64|66|68|70|72|74|76|78|80|82|84|86|88|90|92|94|96|98|100|102|104|106|108|row =  52
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|38|40|42|44|46|48|50|52|54|56|58|60|62|64|66|68|70|72|74|76|78|80|82|84|86|88|90|92|94|96|98|100|102|104|106|108|row =  54
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|38|40|42|44|46|48|50|52|54|56|58|60|62|64|66|68|70|72|74|76|78|80|82|84|86|88|90|92|94|96|98|100|102|104|106|108|row =  56
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|38|40|42|44|46|48|50|52|54|56|58|60|62|64|66|68|70|72|74|76|78|80|82|84|86|88|90|92|94|96|98|100|102|104|106|108|row =  58
0|2|4|6|8|10|12|14|16|18|20|22|24|26|28|30|32|34|36|38|40|42|44|46|48|50|52|54|56|5

In [6]:

total_time = 0
total_mac_cnt = 0
for i in range(len(w)):
    total_time = total_time + compute_time[i]
print("total_time is ", total_time)

total_time is  9.324542999267578
